In [39]:
%load_ext autoreload
%autoreload 2
# for development

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict

In [41]:
from data_definitions import *
# coll, common_tables, ids, nids, content_cols, cols, uniq_per_table, tblregister

In [42]:
from help_functions import * 
# sup, deldupids, my_conv, try_padding, makedate_from_givendate, get_unique_lower, case_insensitive_unique_list, replace_ids


## steps

1. get old tables from mysql database (this could be substituted with filesystem). Note: there are different databases for each period
2. which tables are common to different databases (they do not all contain the same tables)
3. which id columns and which content columns do the tables contain (once again, not all original databases are exactly the same)
4. merge common tables and deduplicate references 
5. dump this intermediate result to disk
6. transformations of the persoon table (checking ids, deduplicate, normalizing dates)
7. transformations of the aanstellingen table (same as step 6)
8. id reference transformations



In [43]:
# step 2
transposed_graph = defaultdict(list)
for node, neighbours in coll.items():
    for neighbour in neighbours:
        transposed_graph[neighbour].append(node)


In [13]:
common_tables

['AcademischeTitel',
 'AdellijkeTitel',
 'aliassen',
 'Bron',
 'BronFunctieDetails',
 'BronRegentDetails',
 'College',
 'Functie',
 'FunctieBovenLokaal',
 'FunctieLokaal',
 'lokaal',
 'provinciaal',
 'Regent',
 'regionaal',
 'stand',
 'BovenLokaalCollegeRegentDetails',
 'Gewest']

In [14]:
# all relevant tables are in common_tables:
# let's put all relevant stuff into one table

common_helptable = {t:{} for t in common_tables}

defs = {'nids':nids,
        'content_cols':content_cols, 
        'cols':cols, 
        'uniq_per_table': uniq_per_table, 
        'tblregister':tblregister
        }

for t in common_helptable:
    ttbl = common_helptable[t]
    for d in defs:
        ttbl[d] = defs[d]

In [15]:
common_helptable

{'AcademischeTitel': {'nids': {'academischetitel': ['idacademischetitel'],
   'adellijketitel': ['idadellijketitel'],
   'aliassen': ['idpersoon'],
   'bron': ['idbron'],
   'bronfunctiedetails': ['idbron', 'idbovenlokaalcollegeregentdetails'],
   'bronregentdetails': ['idregent', 'idbron'],
   'college': ['idcollege', 'id'],
   'functie': ['idfunctie', 'id'],
   'functiebovenlokaal': ['id functiebovenlokaal'],
   'functielokaal': ['id functielokaal'],
   'lokaal': ['idlokaal'],
   'provinciaal': ['idprovincie'],
   'regent': ['idregent', 'idadellijketitel', 'idacademischetitel'],
   'regionaal': ['idregio', 'idregio'],
   'stand': ['idstand'],
   'bovenlokaalcollegeregentdetails': ['id',
    'idregent',
    'idfunctie',
    'idcollege',
    'lokaal',
    'provinciaal',
    'regio',
    'stand',
    'vertegenwoordigend'],
   'gewest': ['idgewest'],
   'data': ['id']},
  'content_cols': {'academischetitel': 'academischetitel',
   'adellijketitel': 'adellijketitel',
   'aliassen': 'alias

In [16]:
common_helptable

{'AcademischeTitel': {'nids': {'academischetitel': ['idacademischetitel'],
   'adellijketitel': ['idadellijketitel'],
   'aliassen': ['idpersoon'],
   'bron': ['idbron'],
   'bronfunctiedetails': ['idbron', 'idbovenlokaalcollegeregentdetails'],
   'bronregentdetails': ['idregent', 'idbron'],
   'college': ['idcollege', 'id'],
   'functie': ['idfunctie', 'id'],
   'functiebovenlokaal': ['id functiebovenlokaal'],
   'functielokaal': ['id functielokaal'],
   'lokaal': ['idlokaal'],
   'provinciaal': ['idprovincie'],
   'regent': ['idregent', 'idadellijketitel', 'idacademischetitel'],
   'regionaal': ['idregio', 'idregio'],
   'stand': ['idstand'],
   'bovenlokaalcollegeregentdetails': ['id',
    'idregent',
    'idfunctie',
    'idcollege',
    'lokaal',
    'provinciaal',
    'regio',
    'stand',
    'vertegenwoordigend'],
   'gewest': ['idgewest'],
   'data': ['id']},
  'content_cols': {'academischetitel': 'academischetitel',
   'adellijketitel': 'adellijketitel',
   'aliassen': 'alias

In [44]:
table_dict = defaultdict(dict)
connection_string = open('connection.txt','r').read()
for table in common_tables:
    for periode in transposed_graph[table]:
        periode = periode.strip().replace(' ', '_')
        tname = '_'.join([periode, table]).strip()
        print(f"getting {tname}")
        dftable = pd.read_sql_table(con=connection_string, table_name=tname)
        dftable = dftable.rename(columns={c:c.lower() for c in dftable.columns})
        dftable.reset_index(inplace=True)
        print(f"adding {table.lower()} to {periode}")
        table_dict[periode][table.lower()] = dftable

getting batfra_AcademischeTitel
adding academischetitel to batfra
getting negentiende_eeuw_AcademischeTitel
adding academischetitel to negentiende_eeuw
getting me_AcademischeTitel
adding academischetitel to me
getting divperioden_AcademischeTitel
adding academischetitel to divperioden
getting republiek_AcademischeTitel
adding academischetitel to republiek
getting batfra_AdellijkeTitel
adding adellijketitel to batfra
getting negentiende_eeuw_AdellijkeTitel
adding adellijketitel to negentiende_eeuw
getting me_AdellijkeTitel
adding adellijketitel to me
getting divperioden_AdellijkeTitel
adding adellijketitel to divperioden
getting republiek_AdellijkeTitel
adding adellijketitel to republiek
getting batfra_aliassen
adding aliassen to batfra
getting negentiende_eeuw_aliassen
adding aliassen to negentiende_eeuw
getting me_aliassen
adding aliassen to me
getting divperioden_aliassen
adding aliassen to divperioden
getting republiek_aliassen
adding aliassen to republiek
getting batfra_Bron
adding

adding bron to divperioden
getting republiek_Bron
adding bron to republiek
getting batfra_BronFunctieDetails
adding bronfunctiedetails to batfra
getting negentiende_eeuw_BronFunctieDetails
adding bronfunctiedetails to negentiende_eeuw
getting me_BronFunctieDetails
adding bronfunctiedetails to me
getting divperioden_BronFunctieDetails
adding bronfunctiedetails to divperioden
getting republiek_BronFunctieDetails
adding bronfunctiedetails to republiek
getting batfra_BronRegentDetails
adding bronregentdetails to batfra
getting negentiende_eeuw_BronRegentDetails
adding bronregentdetails to negentiende_eeuw
getting me_BronRegentDetails
adding bronregentdetails to me
getting divperioden_BronRegentDetails
adding bronregentdetails to divperioden
getting republiek_BronRegentDetails
adding bronregentdetails to republiek
getting batfra_College
adding college to batfra
getting negentiende_eeuw_College
adding college to negentiende_eeuw
getting me_College
adding college to me
getting divperioden_Col

In [18]:
common_tables

['AcademischeTitel',
 'AdellijkeTitel',
 'aliassen',
 'Bron',
 'BronFunctieDetails',
 'BronRegentDetails',
 'College',
 'Functie',
 'FunctieBovenLokaal',
 'FunctieLokaal',
 'lokaal',
 'provinciaal',
 'Regent',
 'regionaal',
 'stand',
 'BovenLokaalCollegeRegentDetails',
 'Gewest']

In [45]:
common_tables = [t.lower() for t in common_tables]

In [20]:
for tbl in common_tables:
    for key in table_dict.keys():
        if tbl in table_dict[key].keys():
            addtbl = table_dict[key][tbl]
        else:
            print (f"no {tbl} in {key}")

no functiebovenlokaal in negentiende_eeuw
no gewest in divperioden


In [46]:
# step 4: join tables
# joined means merged as in making one table from different tables
# we first change the ids and reference ids to old ids and add the table origin to the id

idmappings = defaultdict(list)
raw_joined_tables = {}
for tbl in common_tables:
    for key in table_dict.keys():
        if tbl in table_dict[key].keys():
            addtbl = table_dict[key][tbl]
        else:
            print (f"no {tbl} in {key}")
        try:
            for idnr in nids.get(tbl):
                addtbl[f'old_{idnr}'] = addtbl[idnr].astype(pd.Int64Dtype()).apply(lambda x: f"{key}_{x}")
                idmappings[f'old_{idnr}'].append(tbl)
        except (KeyError, TypeError):
            print (f"no {idnr} on {tbl} in {key}")
        if tbl in raw_joined_tables.keys():
            print(f"joining {key}, {tbl}")
            raw_joined_tables[tbl] = pd.concat([raw_joined_tables[tbl],addtbl])
        else:
            raw_joined_tables[tbl] = addtbl
        
            # joined_tables[tbl].reset_index(inplace=True, drop=True)

joining negentiende_eeuw, academischetitel
joining me, academischetitel
joining divperioden, academischetitel
joining republiek, academischetitel
joining negentiende_eeuw, adellijketitel
joining me, adellijketitel
joining divperioden, adellijketitel
joining republiek, adellijketitel
joining negentiende_eeuw, aliassen
joining me, aliassen
joining divperioden, aliassen
joining republiek, aliassen
joining negentiende_eeuw, bron
joining me, bron
joining divperioden, bron
joining republiek, bron
joining negentiende_eeuw, bronfunctiedetails
joining me, bronfunctiedetails
joining divperioden, bronfunctiedetails
joining republiek, bronfunctiedetails
joining negentiende_eeuw, bronregentdetails
joining me, bronregentdetails
joining divperioden, bronregentdetails
joining republiek, bronregentdetails
joining negentiende_eeuw, college
joining me, college
joining divperioden, college
joining republiek, college
joining negentiende_eeuw, functie
joining me, functie
joining divperioden, functie
joining

In [47]:
# step 4: deduplicate common tables
joined_tables = {}
for tablename in common_tables:
    tbl = raw_joined_tables[tablename]
    val_column = content_cols.get(tablename) or ''
    id_cols = nids[tablename]
    try:
        idc = [i for i in id_cols if i.find(val_column)>-1][0]
    except (IndexError, KeyError):
        # a bit of a lame fallback for the aliassen table
        idc = id_cols[0]
    oldidc = f'old_{idc}'
    if val_column in tbl.columns:
        r = deldupids(tbl, val_column, oldidc)
    else:
        r = tbl
    joined_tables[tablename] = r
    # else:
    #     print('fout', tablename, tbl.columns, oldidc)
    

In [48]:
# step 4a: add aanstelling and person tables to joined_tables 
# and we remove old tables from the dictionary, just to be sure

joined_tables['aanstelling'] = raw_joined_tables.pop('bovenlokaalcollegeregentdetails')
joined_tables['persoon'] = raw_joined_tables.pop('regent')

In [49]:
# step 5: dump the joined tables to disk
# note these tables are still not cleaned up

ids = nids # to keep things confused

try:
    os.makedirs(outdir)
except FileExistsError:
    pass
for key in joined_tables.keys():
    dfout = joined_tables[key]
    dfout.to_csv(os.path.join(outdir, key + '.csv'))
    

In [50]:
# step 6: transformations of the persoon table
# we still need to set a new id on regent 
persoon = joined_tables['persoon']
persoon["persoon_id"] = persoon.index + 1 
persoon.reset_index(inplace=True, drop=True)


In [51]:
# step 6: check if ids are duplicated
persoon.old_idregent.value_counts().sort_values(ascending=False)

batfra_319               1
negentiende_eeuw_6216    1
batfra_2392              1
batfra_797               1
republiek_1429           1
                        ..
republiek_1584           1
negentiende_eeuw_6578    1
me_398                   1
republiek_3116           1
me_294                   1
Name: old_idregent, Length: 15775, dtype: int64

In [52]:
# step 6: remove duplicates
duplicates = persoon.loc[(persoon[cols].duplicated(keep=False))].sort_values('geslachtsnaam')

In [53]:
# step 6 remove duplicates
xx = duplicates.merge(duplicates, left_on=cols,right_on=cols, suffixes=('l', 's'))

In [54]:
# step 6: there are duplicate ids that have to be cleaned up
keypairs = xx.loc[xx.idregentl != xx.idregents][['idregentl' ,'idregents']]
dup = persoon.loc[persoon.duplicated(cols)]
zz = keypairs.loc[keypairs.idregentl.isin(dup.idregent)]


In [55]:
# step 6: continue cleaning up
persoon[cols].drop_duplicates(inplace=True)
persoon.reset_index(drop=True, inplace=True)


/var/folders/9y/s7573g1n3q9fz78c71ryxj4m0000gn/T/ipykernel_47111/3962888377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  persoon[cols].drop_duplicates(inplace=True)


In [56]:
# step 6: more person table cleaning up 
# (the NAs appear as NA in the resulting web representation)


persoon.heerlijkheid.fillna('', inplace=True)
persoon.heerlijkheid2.fillna('', inplace=True)
persoon.heerlijkheid = persoon.apply(lambda x: x.heerlijkheid + ' en ' + x.heerlijkheid2  if x.heerlijkheid2 !='' else x.heerlijkheid, axis=1)

# step 6: normalize dates
# we replace overlijdensjaar with ? as last digit with 9
# that is the only real edit, all other non-standard date elements are replaced
persoon.overlijdensjaar.replace('?','9', inplace=True)

# step 6: more normalizing dates
for j in ["geboortedag", "geboortemaand", "geboortejaar", 
          "overlijdensdag", "overlijdensmaand", "overlijdensjaar"]:
    persoon[j] = pd.to_numeric(persoon[j], errors="coerce")

# step 6: there is a lot of dates that need to be normalized
persoon["geboortedatum_als_bekend"] = persoon.apply(lambda x: '-'.join([try_padding(d) for d in [
                        x.geboortedag, x.geboortemaand, x.geboortejaar ] if d and not np.isnan(d)]),
                        axis=1)

# the d==d is a workaround for all None and NaN, as they are the only items that have no identity
persoon["overlijdensdatum_als_bekend"] = persoon.apply(lambda x: '-'.join([try_padding(d) for d in [
                        x.overlijdensdag, x.overlijdensmaand, x.overlijdensjaar ] if d and d==d]),
                        axis=1)


# step 6: date normalizing
persoon["geboortedatum"] = persoon.geboortedatum_als_bekend.apply(lambda x: makedate_from_givendate(x,start=True))
persoon["overlijdensdatum"] = persoon.overlijdensdatum_als_bekend.apply(lambda x: makedate_from_givendate(x,start=False))



In [57]:
# step 6 NaN filling for some fields
persoon.tussenvoegsel.fillna('', inplace=True)
persoon.geslachtsnaam.fillna('', inplace=True) # gross
persoon['searchable'] = persoon[["tussenvoegsel", "geslachtsnaam"]].apply(lambda x: ' '.join(x), axis=1).str.strip()


In [35]:
# step 6: test for birth dates
persoon[['geboortedatum_als_bekend', 'geboortedatum']].sample(25)

,geboortedatum_als_bekend,geboortedatum
3760,29-06-1753,1753-06-29
12585,1609,1609-01-01
10426,,None
9619,,None
22,1749,1749-01-01
7448,1467,1467-01-01
13476,,None
7959,,None
7242,1442,1442-01-01
12311,1634,1634-01-01


In [58]:
# step 6: we add this to the joined_tables
persoon.reset_index(drop=True, inplace=True)
joined_tables['persoon'] = persoon
#regent.searchable

In [59]:
# step 7: transformations of aanstelling table 
aanstelling = joined_tables['aanstelling']

In [60]:
# step 7: cleaning up strange dates with technical brute force

# we tried to do this with more heuristics, but in the end we just edit out faulty stuff as we do not know
# what to make of it anyway
for d in ['begindag', 'beginmaand', 'beginjaar', 'einddag','eindmaand','eindjaar']:
    try:
        #blcrd[d] = blcrd[d].str.replace(r'[^0-9]+','',regex=True)
        #blcrd[d] = blcrd[d].str.replace(r'[\)\<\-\?\/\>]|Jaco|Duve|Leid|Clee|Will|Krus','',regex=True)
        aanstelling[d] = pd.to_numeric(aanstelling[d], errors='coerce') # this automatically kicks out mistakes in dates
    except AttributeError:
        pass

In [61]:
# step 7: another round of date mangling for aanstelling table
aanstelling["van_als_bekend"] = aanstelling.apply(lambda x: '-'.join([try_padding(d) for d in [
                        x.begindag, x.beginmaand, x.beginjaar ] if d and not np.isnan(d)]),
                        axis=1)

aanstelling["tot_als_bekend"] = aanstelling.apply(lambda x: '-'.join([try_padding(d) for d in [
                        x.einddag, x.eindmaand, x.eindjaar] if d and d==d]),
                        axis=1)

In [62]:
# step 7: renaming
for item in ['van', 'tot']:
    if item == 'tot':
        start = False
    else:
        start=True
    aanstelling[item] = aanstelling[f"{item}_als_bekend"].apply(lambda x: makedate_from_givendate(x,start=start))
    aanstelling[item] = aanstelling[item].astype(str)
   

In [63]:
# step 7: add to joined_tables 

joined_tables['aanstelling'] = aanstelling

## reference replacement

- gegeven een tbl met daarin titel, titelid, oude titel willen we hebben unieke titel, unieke id, referentie naar oude titel
- dus idealiter oude titel  -> nieuwe id, nieuwe titel
- daarom moeten we hebben unieke lijst van titels
- daaruit genereren we een unieke lijst van titels + id
- we voegen die dan toe aan de tabel waarin die titel wordt gerefereerd

In [64]:
# step 8: 
# we have a number of tables that are referenced

idmappings = {i:list(set(idmappings[i])) for i in idmappings.keys()}
#idmappings

In [65]:
# reverse map the ids and references we have to change
graph=idmappings
transposed_graph = defaultdict(list)
for node, neighbours in graph.items():
    for neighbour in neighbours:
        transposed_graph[neighbour.lower()].append(node.lower())
#transposed_graph

In [81]:
[r for t in tblregister.keys() for r in tblregister[t]['reftables']]

['academischetitel',
 'adellijketitel',
 'persoon',
 'bron',
 'aanstelling',
 'bron',
 'persoon',
 'college',
 'functie',
 'lokaal',
 'provinciaal',
 'regionaal',
 'stand',
 'persoon']

In [66]:
reftables = [r for t in tblregister.keys() for r in tblregister[t]['reftables']]
list(set(reftables))

['functie',
 'provinciaal',
 'persoon',
 'college',
 'lokaal',
 'academischetitel',
 'aanstelling',
 'stand',
 'regionaal',
 'bron',
 'adellijketitel']

In [67]:
# this function now converts selected columns like function and titles to lowercase 

clean_references = {}


In [68]:
references = {}
for tbln in reftables:    
    ntbln = get_unique_lower(tbln, joined_tables, uniq_per_table, clean_references)
    ntbln.reset_index(inplace=True, drop=True)
    references[tbln] = ntbln
    print (f"updated {tbln}")
    


updated academischetitel
updated adellijketitel
updated persoon
updated bron
updated aanstelling
updated bron
updated persoon
updated college
updated functie
updated lokaal
updated provinciaal
updated regionaal
updated stand
updated persoon


In [69]:
updatetables = ['persoon', 
                'aliassen',  
                'bronregentdetails', 
                'bronfunctiedetails',
                'aanstelling',
                ]

In [80]:
nwetabellen = {}
for tbln in tblregister.keys():
    if tbln in updatetables:
        worktable = references.get(tbln)
        if worktable is None:
            worktable = joined_tables[tbln]
        for reference in tblregister[tbln]['reftables'].keys():
            rtbl = reference
            referencecolumn = tblregister[tbln]['reftables'][reference]
            
            workreference = references.get(rtbl)

            #print ('workreference: ', workreference)
            if workreference is not None:
                column = uniq_per_table[rtbl]['id']
                uniqcolumn = uniq_per_table[rtbl]['uniq']
                print (f"updating {tbln} with {rtbl}, columns: {column},{referencecolumn}!")
                columnlist = list(worktable.columns)+[f'{rtbl}_id']
                #if tbln=='bovenlokaalcollegeregentdetails':
                    #print (columnlist)
                worktable = worktable.merge(workreference[[column, f'{rtbl}_id']], left_on=referencecolumn, right_on=column, how='left')
                mclist = [c for c in columnlist if c in worktable.columns]
                updatedworktable = worktable[mclist]
            if rtbl not in nwetabellen.keys():
                nwetabellen[rtbl] = references[rtbl]
            worktable = updatedworktable
            worktable.reset_index(drop=True, inplace=True)
        nwetabellen[tbln] = worktable
        print('nwetabellen updated with ', tbln)
    else:
        nwetabellen[tbln] = references[tbln]

# we need to fix some more

updating persoon with academischetitel, columns: old_idacademischetitel,old_idacademischetitel!
updating persoon with adellijketitel, columns: old_idadellijketitel,old_idadellijketitel!
nwetabellen updated with  persoon
updating aliassen with persoon, columns: old_idregent,old_idpersoon!
nwetabellen updated with  aliassen
updating bronfunctiedetails with bron, columns: old_idbron,old_idbron!
updating bronfunctiedetails with aanstelling, columns: old_id,old_idbovenlokaalcollegeregentdetails!
nwetabellen updated with  bronfunctiedetails
updating bronregentdetails with bron, columns: old_idbron,old_idbron!
updating bronregentdetails with persoon, columns: old_idregent,old_idregent!
nwetabellen updated with  bronregentdetails
updating aanstelling with college, columns: old_idcollege,old_idcollege!
updating aanstelling with functie, columns: old_idfunctie,old_idfunctie!
updating aanstelling with lokaal, columns: old_idlokaal,old_lokaal!
updating aanstelling with provinciaal, columns: old_id

In [82]:
nwetabellen['aanstelling'].reset_index(inplace=True, drop=True)
nwetabellen['aanstelling']['id'] = nwetabellen['aanstelling'].index


In [83]:
nwetabellen['aanstelling'][['college_id', 'functie_id', 'lokaal_id', 'persoon_id', 'old_idcollege', 'old_idfunctie', 'old_idregent']]

,college_id,functie_id,lokaal_id,persoon_id,old_idcollege,old_idfunctie,old_idregent
0,90.0,43.0,NaN,242.0,batfra_127,batfra_92,batfra_335
1,436.0,NaN,NaN,6726.0,republiek_9,republiek_1,republiek_22
2,144.0,114.0,NaN,242.0,batfra_198,batfra_326,batfra_335
3,408.0,409.0,NaN,1.0,me_1,me_1,me_1
4,438.0,NaN,NaN,6726.0,republiek_11,republiek_1,republiek_22
...,...,...,...,...,...,...,...
37671,520.0,NaN,NaN,5636.0,republiek_162,republiek_50,republiek_7066
37672,520.0,460.0,NaN,3946.0,republiek_162,republiek_85,republiek_4567
37673,520.0,NaN,NaN,6090.0,republiek_162,republiek_50,republiek_7062
37674,520.0,NaN,NaN,4722.0,republiek_162,republiek_50,republiek_5777


In [84]:
aanstelling = nwetabellen['aanstelling']
persoon = nwetabellen['persoon']

In [85]:
patchtbl = idmappings['old_idregent']
# idmappings

In [86]:
for tbl in patchtbl:
    try:
        ptbl = nwetabellen[tbl]
    except KeyError:
        replkey = {'regent':'persoon',
                   'bovenlokaalcollegeregentdetails':'aanstelling'}
        tbl = replkey[tbl]   
        ptbl = nwetabellen[tbl]
    updated = ptbl.merge(zz, how='left', left_on='persoon_id', right_on='idregents', suffixes=('', '_new'))
    updated['persoon_id'] = np.where(pd.notnull(updated['idregentl']), updated['idregentl'], updated['persoon_id'])
    updated.drop(['idregents', 'idregentl'], axis=1, inplace=True)
    print (f"updated {tbl}")

updated persoon
updated bronregentdetails
updated aanstelling


In [89]:
# a bit of checking. If the values in the two tables are different this throws an assertion error
compr = aanstelling.persoon_id.value_counts() == nwetabellen['aanstelling'].persoon_id.value_counts()
assert(len(compr.loc[~compr])==0)

In [90]:
nwetabellen['persoon'].columns

Index(['persoon_id', 'id', 'index', 'idregent', 'geslachtsnaam',
       'tussenvoegsel', 'idadellijketitel', 'idacademischetitel', 'voornaam',
       'adel', 'adelspredikaat', 'heerlijkheid', 'heerlijkheid2',
       'geboortejaar', 'onbepaaldgeboortedatum', 'overlijdensjaar',
       'onbepaaldoverlijdensdatum', 'opmerkingen', 'periode', 'eindcontrole',
       'old_idregent', 'old_idadellijketitel', 'old_idacademischetitel',
       'geboortedag', 'geboortemaand', 'doopjaar', 'geboorteplaats',
       'overlijdensdag', 'overlijdensmaand', 'overlijdensplaats', 'tempid',
       'opmerkingen2', 'geboortedatum_als_bekend',
       'overlijdensdatum_als_bekend', 'geboortedatum', 'overlijdensdatum',
       'searchable', 'academischetitel_id', 'adellijketitel_id'],
      dtype='object')

In [91]:
#some renaming to be sure
persoon = nwetabellen['persoon']
persoon.rename(columns={
        'heerlijkheid':'heerlijkheid',
        'voornaam':'voornaam',
        'tussenvoegsel':'tussenvoegsel',
        'geslachtsnaam':'geslachtsnaam',
        'searchable_geslachtsnaam':'searchable',
        'geboortedatum':'geboortedatum',
        'geboortedatum_als_bekend':'geboortedatum_als_bekend',
        'geboorteplaats':'geboorteplaats',
        'doopjaar':'doopjaar',
        'overlijdensdatum':'overlijdensdatum',
        'overlijdensdatum_als_bekend':'overlijdensdatum_als_bekend',
        'overlijdensplaats':'overlijdensplaats',
        'adellijketitel_id':'adellijketitel_id',
        'academischetitel_id':'academischetitel_id',
        'opmerkingen':'opmerkingen',
        }, inplace=True)
nwetabellen['persoon'] = persoon

In [92]:
persoon.columns

Index(['persoon_id', 'id', 'index', 'idregent', 'geslachtsnaam',
       'tussenvoegsel', 'idadellijketitel', 'idacademischetitel', 'voornaam',
       'adel', 'adelspredikaat', 'heerlijkheid', 'heerlijkheid2',
       'geboortejaar', 'onbepaaldgeboortedatum', 'overlijdensjaar',
       'onbepaaldoverlijdensdatum', 'opmerkingen', 'periode', 'eindcontrole',
       'old_idregent', 'old_idadellijketitel', 'old_idacademischetitel',
       'geboortedag', 'geboortemaand', 'doopjaar', 'geboorteplaats',
       'overlijdensdag', 'overlijdensmaand', 'overlijdensplaats', 'tempid',
       'opmerkingen2', 'geboortedatum_als_bekend',
       'overlijdensdatum_als_bekend', 'geboortedatum', 'overlijdensdatum',
       'searchable', 'academischetitel_id', 'adellijketitel_id'],
      dtype='object')

In [93]:
# a bit of checking. If the values in the two tables are different this throws an assertion error
compp = persoon.persoon_id == persoon.id
assert(len(compp.loc[~compp]) == 0)

In [95]:
#some more renaming to be sure
aanstelling = nwetabellen['aanstelling']
aanstelling.rename(columns={'id': 'id',
 'begindag': 'begindag',
 'beginjaar': 'beginjaar',
 'beginmaand': 'beginmaand',
 'einddag': 'einddag',
 'eindjaar': 'eindjaar',
 'eindmaand': 'eindmaand',
 #'idcollege': 'idcollege',
 #'idfunctie': 'idfunctie',
 #'idregent': 'idregent',
 #'index': 'index',
 #'lokaal': 'lokaal',
 #'old_id': 'old_id',
 'old_idcollege': 'old_idcollege',
 'old_idfunctie': 'old_idfunctie',
 'old_idregent': 'old_idregent',
 'old_lokaal': 'old_lokaal',
 'old_provinciaal': 'old_provinciaal',
 'old_regio': 'old_regio',
 'old_stand': 'old_stand',
 'old_vertegenwoordigend': 'old_vertegenwoordigend',
 'opmerkingen': 'opmerkingen',
 'periode': 'periode',
 'vertegenwoordigend': 'vertegenwoordigend',
 'college_id': 'college_id',
 'functie_id': 'functie_id',
 'lokaal_id': 'lokaal_id',
 'provinciaal_id': 'provincie_id',
 'regionaal_id': 'regio_id',
 'stand_id': 'stand_id',
 'persoon_id': 'persoon_id'}, inplace=True)
nwetabellen['aanstelling'] = aanstelling

In [96]:
aanstelling.columns

Index(['id', 'index', 'periode', 'idregent', 'idfunctie', 'idcollege',
       'begindag', 'beginmaand', 'beginjaar', 'einddag', 'eindmaand',
       'eindjaar', 'vertegenwoordigend', 'provinciaal', 'opmerkingen', 'stand',
       'lokaal', 'regio', 'old_id', 'old_idregent', 'old_idfunctie',
       'old_idcollege', 'old_lokaal', 'old_provinciaal', 'old_regio',
       'old_stand', 'old_vertegenwoordigend', 'van_als_bekend',
       'tot_als_bekend', 'van', 'tot', 'aanstelling_id', 'college_id',
       'functie_id', 'lokaal_id', 'provincie_id', 'regio_id', 'stand_id',
       'persoon_id'],
      dtype='object')

In [97]:
exporttabellen = {'academischetitel':'academische_titel', 
                  'adellijketitel':'adellijke_titel', 
                  'aliassen':'alias', 
                  'bron':'bron', 
                  'bronregentdetails':'bron_details', 
                  'college':'instelling', 
                  'functie':'functie', 
                  'lokaal':'lokaal', 
                  'provinciaal':'provincie', 
                  'regionaal':'regio', 
                  'stand':'stand',
                  'gewest':'gewest',
                  'aanstelling':'aanstelling',
                  'persoon':'persoon'}

In [98]:
# pandas cannot handle period in to_sql so we convert it to text

for column in ['geboortedatum', 'overlijdensdatum']:
    persoon[column] = persoon[column].astype('str')
    persoon[column] = persoon[column].apply(lambda x: 'Null' if isinstance(x, str) and x in ['','NaT','Null','None'] else x)
    
for column in ['van', 'tot']:
    #aanstelling[column] = aanstelling[column].fillna(0)
    aanstelling[column] = aanstelling[column].astype('str')
    aanstelling[column] = aanstelling[column].apply(lambda x: 'Null' if isinstance(x, str) and x in ['','NaT','Null','None'] else x)

In [99]:
#another check
assert(len(aanstelling.loc[aanstelling.tot == 'None'])==0)

In [100]:
for tbl in nwetabellen.keys():
    if tbl in clean_references.keys():
        nwetabellen[tbl] = clean_references[tbl]

In [101]:
# and a bit of name cleanup
columnmaps={
"academischetitel": {
    'academischetitel':'naam',
    'old_idacademischetitel':'old_idacademischetitel',
    'academischetitel_id':'id',
    'id':'id'},

"adellijketitel": {
    'adellijketitel':'naam',
    'old_idadellijketitel':'old_idadellijketitel',
    'adellijketitel_id':'id',
    'id':'id'},

"aliassen": {
    'alias':'naam',
    'old_idpersoon':'old_idpersoon',
    'regent_id':'persoon_id',
    'id':'id'},

"bron": {
    'index':'id',
    'bron':'naam',
    'old_idbron':'old_idbron',
    'bron_id':'id',
    'id':'id'},

"bronregentdetails": {
    'id':'id',
    'deel en paginanummer':'details',
    'old_idregent':'old_idregent',
    'old_idbron':'old_idbron',
    'bron_id':'bron_id',
    'regent_id':'regent_id'},

"college": {
    'college':'naam',
    'periode':'periode',
    'old_idcollege':'old_idcollege',
    'college_id':'id',
    'id':'id'},

"functie": {
    'functie':'naam',
    'periode':'periode',
    'lokaal':'lokaal',
    'old_idfunctie':'old_idfunctie',
    'functie_id':'id',
    'id':'id'},

"lokaal": {
    'lokaal':'naam',
    'old_idlokaal':'old_idlokaal',
    'lokaal_id':'id',
    'id':'id'},

"provinciaal": {
    'provincie':'naam',
    'old_idprovincie':'old_idprovincie',
    'provinciaal_id':'id',
    'id':'id'},

"regionaal": {
    'regio':'naam',
    'old_idregio':'old_idregio',
    'regionaal_id':'id',
    'id':'id'},

"stand": {
    'stand':'naam',
    'old_idstand':'old_idstand',
    'stand_id':'stand_id',
    'id':'id'},

"persoon": {
    'adel':'adel',
    'adelspredikaat':'adelspredikaat',
    'doopjaar':'doopjaar',
    'eindcontrole':'eindcontrole',
    'geboortedag':'geboortedag',
    'geboortejaar':'geboortejaar',
    'geboortemaand':'geboortemaand',
    'geboorteplaats':'geboorteplaats',
    'geslachtsnaam':'geslachtsnaam',
    'heerlijkheid':'heerlijkheid',
    'old_idacademischetitel':'old_idacademischetitel',
    'old_idadellijketitel':'old_idadellijketitel',
    'old_idregent':'old_idregent',
    'opmerkingen':'opmerkingen',
    'overlijdensdag':'overlijdensdag',
    'overlijdensjaar':'overlijdensjaar',
    'overlijdensmaand':'overlijdensmaand',
    'overlijdensplaats':'overlijdensplaats',
    'periode':'periode',
    'tussenvoegsel':'tussenvoegsel',
    'voornaam':'voornaam',
    #'regent_id':'id',
    'id':'id',
    'academischetitel_id':'academischetitel_id',
    'adellijketitel_id':'adellijketitel_id',
    'geboortedatum':'geboortedatum',
    'overlijdensdatum':'overlijdensdatum',
    'geboortedatum':'geboortedatum',
    'overlijdensdatum':'overlijdensdatum',
    'searchable':'searchable'},

"aanstelling": {
    'begindag':'begindag',
    'beginjaar':'beginjaar',
    'beginmaand':'beginmaand',
    'einddag':'einddag',
    'eindjaar':'eindjaar',
    'eindmaand':'eindmaand',
    'id':'id',
    'old_id':'old_id',
    'old_idcollege':'old_idcollege',
    'old_idfunctie':'old_idfunctie',
    'old_idregent':'old_idregent',
    'old_lokaal':'old_lokaal',
    'old_provinciaal':'old_provinciaal',
    'old_regio':'old_regio',
    'old_stand':'old_stand',
    'old_vertegenwoordigend':'old_vertegenwoordigend',
    'opmerkingen':'opmerkingen',
    'periode':'periode',
    'vertegenwoordigend':'vertegenwoordigend',
    'college_id':'instelling_id',
    'functie_id':'functie_id',
    'lokaal_id':'lokaal_id',
    'provinciaal_id':'provincie_id',
    'regionaal_id':'regio_id',
    'stand_id':'stand_id',
    #'regent_id':'persoon_id',
    'persoon_id':'persoon_id',
    'van_als_bekend':'van_als_bekend',
    'tot_als_bekend':'tot_als_bekend',
    'van':'van',
    'tot':'tot'},
    
    'bronfunctiedetails':{
    'idbron':'idbron', 
    'idbovenlokaalcollegeregentdetails':'idaanstelling', 
    'old_idbron':'old_idbron',
    'old_idbovenlokaalcollegeregentdetails':'old_idbovenlokaalcollegeregentdetails', 
    'bron_id':'bron_id', 
    'aanstelling_id':'aanstelling_id'}
    }



In [102]:
extab = {}
for tbl in nwetabellen:
    t = nwetabellen[tbl]
    if tbl in clean_references.keys():
        if tbl not in ['persoon', 'aanstelling']:
            t = clean_references[tbl]
    target = columnmaps[tbl]
    extab['aanstelling'] = aanstelling
    extab['persoon'] = persoon
    if target:
        xtab = t.rename(columns=target)
        drops = [c for c in t.columns if c not in target.keys()]
        tabn = exporttabellen.get(tbl) or ''
        if 'naam' in xtab.columns:
            xtab['naam'].fillna('', inplace=True)
        extab[tabn] = xtab.drop(columns=drops)


In [106]:
# load reference tables first
torder = ['academische_titel', 
          'adellijke_titel',  
          'bron',
          'instelling', 
          'functie', 
          'lokaal', 
          'provincie', 
          'regio', 
          'stand', 
          'persoon', 
          'alias',
          #'aanstelling', 
          'bron_details']

In [110]:
extab['persoon'] = extab['persoon'].replace('NaT', pd.NA)
extab['persoon'] = extab['persoon'].replace('Null', pd.NA)

## Write to Database

In [124]:
import json
with open('connection.json','r') as infl:
    con = json.load(infl)
    
connection_string = con['raa_out']

In [125]:
import sys, subprocess, os
from io import StringIO
import pandas as pd
VERBOSE = True

from sqlalchemy import create_engine, inspect
from sqlalchemy.schema import CreateSchema

engine = create_engine(connection_string, echo=False)

In [126]:
#from https://stackoverflow.com/questions/50927740/sqlalchemy-create-schema-if-not-exists
from sqlalchemy_utils.functions import database_exists, create_database

if not database_exists(connection_string):
    create_database(connection_string)

In [117]:


def mdb_to_sql(engine, database_path, prefix='',basedir='.'):
    """via pandas"""
    subprocess.call(["mdb-schema", database_path, "mysql"])
    # Get the list of table names with "mdb-tables"
    table_names = subprocess.Popen(["mdb-tables", "-1", database_path],
                                   stdout=subprocess.PIPE).communicate()[0]
    tables = table_names.splitlines()
    sys.stdout.flush()
    # Dump each table as a stringio using "mdb-export",
    out_tables = {}
    prefix = prefix.replace(' ', '_') + '_'
    for rtable in tables:
        table = rtable.decode()
        if VERBOSE: print('running table:',table)
        if table != '':
            tname = prefix+table
            if VERBOSE: print("Dumping " + table)
            contents = subprocess.Popen(["mdb-export", database_path, table],
                                        stdout=subprocess.PIPE).communicate()[0]
            temp_io = StringIO(contents.decode("utf8")) # not f*cking latin1
            #print(table, temp_io)
            table = pd.read_csv(temp_io)
            try:
                table.to_sql(con=engine, name=tname, index=False, if_exists="replace")
                t_out_name = os.path.join(basedir,'mdbdump',tname)+'.csv'
                print('outfile', t_out_name)
                table.to_csv(t_out_name,sep="\t")
                print ("written: ", tname)
                done.append(tname)
            except:
                print('fout', tname)
                # table.to_sql(con=engine, name=tname, index=False, if_exists="replace")
                # t_out_name = os.path.join(basedir,'mdbdump',tname)+'.csv'
                # table.to_csv(t_out_name,sep="\t")


In [127]:
engine = create_engine(connection_string, echo=False)

for tbl in torder:
    table = extab[tbl]
    table.to_sql(con=engine, name=tbl, index=False, if_exists="append")


In [128]:
# we do aanstelling separately because duplicate ids cause too much trouble and we do not need them for anything anyway
aanstelling = extab['aanstelling']
truncated_aanstelling = aanstelling[[c for c in aanstelling.columns if c not in ['id']]]

In [129]:
for column in ['van', 'tot']:
    #aanstelling[column] = aanstelling[column].fillna(0)
    truncated_aanstelling[column] = aanstelling[column].astype('str')
    truncated_aanstelling[column] = aanstelling[column].apply(lambda x: pd.NA if isinstance(x, str) and x in ['','NaT','Null','None'] else x)

/var/folders/9y/s7573g1n3q9fz78c71ryxj4m0000gn/T/ipykernel_47111/4090701380.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truncated_aanstelling[column] = aanstelling[column].astype('str')
/var/folders/9y/s7573g1n3q9fz78c71ryxj4m0000gn/T/ipykernel_47111/4090701380.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truncated_aanstelling[column] = aanstelling[column].apply(lambda x: pd.NA if isinstance(x, str) and x in ['','NaT','Null','None'] else x)


In [130]:
truncated_aanstelling.to_sql(con=engine, name='aanstelling', index=False, if_exists="append")